In [234]:
import pandas as pd
import numpy as np
import nltk
import spacy
import difflib
nltk.download('stopwords')
from tqdm import tqdm
from text_to_num import text2num
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer
tqdm.pandas(desc="my bar!")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheusmbr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [235]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['DS_RESUMO', 'DS_PALAVRA_CHAVE', 'NM_AREA_AVALIACAO']].head(100).copy()

In [236]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 0:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [237]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [238]:
df_processed.area_avaliacao = df_processed.area_avaliacao.progress_apply(rename_areas)
del rename_areas

my bar!: 100%|██████████| 100/100 [00:00<00:00, 233796.21it/s]


In [239]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [240]:
df_processed['colegios'] = ''
df_processed['areas'] = ''
df_processed.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

for colegios, areas in colegios_dict.items():
    for areas, subareas in areas.items():
        df_processed.loc[df_processed.subareas.isin(subareas), 'colegios'] = colegios
        df_processed.loc[df_processed.subareas.isin(subareas), 'areas'] = areas
        
df_processed = df_processed[['resumo', 'palavra_chave', 'colegios', 'areas', 'subareas']]

In [241]:
df_processed.palavra_chave.replace(to_replace = r'1[.]', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' \d[.]', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'(\S{2,})(?:\.)', value = '\\1;', regex=True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'\b(pt|en)\b', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' ,|, | [,] | /|/ | [/] ', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' [.] |\(\d\)', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' \x96 |\x93', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'[^a-z0-9\s\x80-\xff,.:;?!-]', value = '', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' - |; | ;|; ;', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r';{2,}', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace= r'[\W_]+$|^[;]+', value='', regex=True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'palavras-chave\S', value = '', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'^\s+|\s+$|\x80', value = '', regex = True, inplace=True)

In [242]:
df_processed.resumo.replace(to_replace= r'[^a-zA-Z0-9\s]+', value='', regex=True, inplace=True)

In [243]:
key_words = df_processed.palavra_chave.copy()
key_words = list(set(';'.join(key_words).split(';')))
key_words = [item for item in key_words if len(item) > 1]

for i in range (len(key_words)):
    key_words[i] = key_words[i].strip()
    
key_words = [item for item in key_words if len(item) < 80]

In [244]:
print(len(key_words))
print(len(set(key_words)))

385
385


In [245]:
def calculate_similarity_or_pertency(sentence: str, text: str) -> str:
    if sentence in text:
        return True
    similaridade = difflib.SequenceMatcher(None, sentence, text).ratio()
    porcentagem = int(similaridade * 100)
    if porcentagem > 80:
        return True
    
    return False

In [246]:
freq_dict = {
    'ciências da vida': {
        'ciências agrárias': {
            'ciência de alimentos': {},
            'ciências agrárias i': {},
            'medicina veterinária': {},
            'zootecnia / recursos pesqueiros': {}
            },
        'ciências biológicas': {
            'biodiversidade': {},
            'ciências biológicas i': {},
            'ciências biológicas ii': {},
            'ciências biológicas iii': {}
            },
        'ciências da saúde': {
            'educação física': {},
            'enfermagem': {},
            'farmácia': {},
            'medicina i': {},
            'medicina ii': {},
            'medicina iii': {},
            'nutrição': {},
            'odontologia': {},
            'saúde coletiva': {}
            }
    },
    'humanidades': {
        'ciências humanas': {
            'antropologia / arqueologia': {},
            'ciência política e relações internacionais': {},
            'ciências da religião e teologia': {},
            'educação': {},
            'filosofia': {},
            'geografia': {},
            'história': {},
            'psicologia': {},
            'sociologia': {}
        },
        'ciências sociais aplicadas': {
            'administração pública e de empresas': {},
            'ciências contábeis e turismo': {},
            'arquitetura': {},
            'urbanismo e design': {},
            'comunicação e informação': {},
            'direito': {},
            'economia': {},
            'planejamento urbano e regional / demografia': {},
            'serviço social': {}
        },
        'linguística, letras e artes': {
            'artes': {},
            'linguística e literatura': {}
        }
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': {
            'astronomia / física': {},
            'ciência da computação': {},
            'geociências': {},
            'matemática / probabilidade e estatística': {},
            'química': {}
        },
        'engenharias': {
            'engenharias i': {},
            'engenharias ii': {},
            'engenharias iii': {},
            'engenharias iv': {}
        },
        'multidisciplinar': {
            'biotecnologia': {},
            'ciências ambientais': {},
            'ensino': {},
            'interdisciplinar': {},
            'materiais': {}
        }
    }
}

In [247]:
# freq = freq_dict.copy()

# for character in key_words:
#     for subarea in sorted(df_processed.subareas.unique()):
#         for area in sorted(df_processed.areas[df_processed.subareas == subarea].unique()):
#             for colegio in sorted(df_processed.colegios[df_processed.areas == area].unique()):
#                 freq[colegio][area][subarea][character] = 0

#     corte = df_processed[df_processed.palavra_chave.apply(lambda x: calculate_similarity_or_pertency(character, x)) | df_processed.resumo.apply(lambda x: calculate_similarity_or_pertency(character, x))].iloc[:, -3:]

#     for subarea in sorted(corte.subareas.unique()):
#         colegio, area = corte[corte.subareas == subarea].colegios.unique()[0], corte[corte.subareas == subarea].areas.unique()[0]
#         freq[colegio][area][subarea][character] = corte.subareas.value_counts()[subarea]

In [248]:
import cupy as cp
import concurrent.futures

freq = freq_dict.copy()

def calculate_frequencies(character):
    for subarea in sorted(df_processed.subareas.unique()):
        for area in sorted(df_processed.areas[df_processed.subareas == subarea].unique()):
            for colegio in sorted(df_processed.colegios[df_processed.areas == area].unique()):
                freq[colegio][area][subarea][character] = 0
    
    corte = calculate_similarity(character)
    
    for subarea in sorted(corte.subareas.unique()):
        colegio, area = corte[corte.subareas == subarea].colegios.unique()[0], corte[corte.subareas == subarea].areas.unique()[0]
        freq[colegio][area][subarea][character] = corte.subareas.value_counts()[subarea]

def calculate_similarity(character):
    keywords = cp.asarray([character] * len(df_processed))
    df_processed_cp = cp.asarray(df_processed[['palavra_chave', 'resumo']].values)

    similarity = cp.sum(df_processed_cp == keywords[:, None], axis=-1)
    return df_processed[similarity > 0]

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    tasks = [executor.submit(calculate_frequencies, character) for character in key_words]
    concurrent.futures.wait(tasks)


In [249]:
data = []

# Iterando pelas chaves do dicionário
for colegio, areas in freq.items():
    for area, subareas in areas.items():
        for subarea, palavras_chave in subareas.items():
            for palavra_chave, frequencia in palavras_chave.items():
                data.append([colegio, area, subarea, palavra_chave, frequencia])

# Criando o dataframe a partir da lista de informações
freq = pd.DataFrame(data, columns=["colegio", "area", "subarea", "palavra_chave", "frequencia"])

In [250]:
freq.set_index('palavra_chave').sort_index()

,colegio,area,subarea,frequencia
palavra_chave,,,,
3 tesla humanos,ciências da vida,ciências da saúde,medicina ii,0
3 tesla humanos,"ciências exatas, tecnológicas e multidisciplinar",engenharias,engenharias i,0
3 tesla humanos,ciências da vida,ciências da saúde,medicina i,0
3 tesla humanos,"ciências exatas, tecnológicas e multidisciplinar",ciências exatas e da terra,ciência da computação,0
3 tesla humanos,ciências da vida,ciências biológicas,biodiversidade,0
...,...,...,...,...
óleos essenciais,ciências da vida,ciências biológicas,biodiversidade,0
óleos essenciais,"ciências exatas, tecnológicas e multidisciplinar",ciências exatas e da terra,ciência da computação,0
óleos essenciais,ciências da vida,ciências agrárias,ciências agrárias i,0
